# Analyze results for 3D CGAN
Feb 22, 2021

In [1]:
import     numpy as np
import matplotlib.pyplot as plt
import pandas as pd

import subprocess as sp
import sys
import os
import glob
import pickle 

from matplotlib.colors import LogNorm, PowerNorm, Normalize
import seaborn as sns
from functools import reduce

In [2]:
from ipywidgets import *

In [3]:
%matplotlib widget

In [4]:
sys.path.append('/global/u1/v/vpa/project/jpt_notebooks/Cosmology/Cosmo_GAN/repositories/cosmogan_pytorch/code/modules_image_analysis/')
from modules_img_analysis import *

In [5]:
sys.path.append('/global/u1/v/vpa/project/jpt_notebooks/Cosmology/Cosmo_GAN/repositories/cosmogan_pytorch/code/5_3d_cgan/1_main_code/')
import post_analysis_pandas as post


In [6]:
### Transformation functions for image pixel values
def f_transform(x):
    return 2.*x/(x + 4.) - 1.

def f_invtransform(s):
    return 4.*(1. + s)/(1. - s)


### Read validation data

In [7]:
# bins=np.concatenate([np.array([-0.5]),np.arange(0.5,20.5,1),np.arange(20.5,100.5,5),np.arange(100.5,1000.5,50),np.array([2000])]) #bin edges to use
bins=np.concatenate([np.array([-0.5]),np.arange(0.5,100.5,5),np.arange(100.5,300.5,20),np.arange(300.5,1000.5,50),np.array([2000])]) #bin edges to use

bins=f_transform(bins)   ### scale to (-1,1)
# ### Extract validation data
sigma_lst=[0.5,0.65,0.8,1.1]
labels_lst=range(len(sigma_lst))
bkgnd_dict={}
num_bkgnd=200

for label in labels_lst:
    fname='/global/cfs/cdirs/m3363/vayyar/cosmogan_data/raw_data/3d_data/dataset2a_3dcgan_4univs_64cube_simple_splicing/norm_1_sig_{0}_train_val.npy'.format(sigma_lst[label])
    print(fname)
    samples=np.load(fname,mmap_mode='r')[-num_bkgnd:][:,0,:,:]
    
    dict_val=post.f_compute_hist_spect(samples,bins)
    bkgnd_dict[str(sigma_lst[label])]=dict_val
    del samples

/global/cfs/cdirs/m3363/vayyar/cosmogan_data/raw_data/3d_data/dataset2a_3dcgan_4univs_64cube_simple_splicing/norm_1_sig_0.5_train_val.npy
/global/cfs/cdirs/m3363/vayyar/cosmogan_data/raw_data/3d_data/dataset2a_3dcgan_4univs_64cube_simple_splicing/norm_1_sig_0.65_train_val.npy
/global/cfs/cdirs/m3363/vayyar/cosmogan_data/raw_data/3d_data/dataset2a_3dcgan_4univs_64cube_simple_splicing/norm_1_sig_0.8_train_val.npy
/global/cfs/cdirs/m3363/vayyar/cosmogan_data/raw_data/3d_data/dataset2a_3dcgan_4univs_64cube_simple_splicing/norm_1_sig_1.1_train_val.npy


## Read data

In [8]:
# main_dir='/global/cfs/cdirs/m3363/vayyar/cosmogan_data/results_from_other_code/pytorch/results/128sq/'
# results_dir=main_dir+'20201002_064327'

In [9]:
dict1={'64':'/global/cfs/cdirs/m3363/vayyar/cosmogan_data/results_from_other_code/pytorch/results/3d_cGAN/',
      '512':'/global/cfs/cdirs/m3363/vayyar/cosmogan_data/results_from_other_code/pytorch/results/3d_cGAN/'}

u=interactive(lambda x: dict1[x], x=Select(options=dict1.keys()))
# display(u)


In [11]:
# parent_dir=u.result
parent_dir=dict1['64']
dir_lst=[i.split('/')[-1] for i in glob.glob(parent_dir+'2021*')]
w=interactive(lambda x: x, x=Dropdown(options=dir_lst))
display(w)

interactive(children=(Dropdown(description='x', options=('20210223_210217_3dcgan_predict_0.8_m2', '20210227_05…

In [12]:
result=w.result
result_dir=parent_dir+result
print(result_dir)

/global/cfs/cdirs/m3363/vayyar/cosmogan_data/results_from_other_code/pytorch/results/3d_cGAN/20210309_200006_3dcgan_predict_0.65_m2


## Plot Losses

In [13]:
df_metrics=pd.read_pickle(result_dir+'/df_metrics.pkle').astype(np.float64)


In [14]:
df_metrics.tail(10)

,step,epoch,Dreal,Dfake,Dfull,G_adv,G_full,spec_loss,hist_loss,spec_chi,hist_chi,D(x),D_G_z1,D_G_z2,time
42174,42174.0,8.0,0.167716,0.144436,0.312152,3.123097,110.863625,107.740524,-0.198246,110.833054,0.074876,3.977784,-3.179225,-3.069629,0.444174
42175,42175.0,8.0,0.137182,0.183805,0.320987,6.201644,115.113647,108.912003,-0.060912,108.550056,0.853538,4.130019,-2.229486,-6.199295,0.440786
42176,42176.0,8.0,0.149297,0.154920,0.304217,5.532454,115.181160,109.648705,0.034498,109.046814,0.048767,3.135159,-5.434539,-5.527760,0.442113
42177,42177.0,8.0,0.152173,0.143449,0.295622,3.958974,114.134972,110.175995,0.056996,108.034409,-0.012792,3.690227,-4.867984,-3.934968,0.440030
42178,42178.0,8.0,0.177096,0.116655,0.293751,4.194742,116.571449,112.376709,-0.251165,108.035934,-0.076118,4.084338,-3.006944,-4.172380,0.446037
42179,42179.0,8.0,0.200175,0.176478,0.376653,4.720345,111.374382,106.654037,0.183944,107.338791,-0.094771,3.722203,-3.014214,-4.708409,0.438840
42180,42180.0,8.0,0.209232,0.146993,0.356225,4.338779,109.090210,104.751434,0.222613,105.626541,0.580116,3.847753,-4.059111,-4.317606,0.442672
42181,42181.0,8.0,0.149167,0.120724,0.269891,4.291422,112.800957,108.509537,0.333497,108.668434,0.435596,3.474184,-3.202874,-4.271730,0.442397
42182,42182.0,8.0,0.162860,0.117658,0.280518,4.281157,114.774017,110.492859,0.253269,108.603165,1.131474,3.287461,-3.426367,-4.258962,0.466042
42183,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,108.712822,0.443179,NaN,NaN,NaN,NaN


In [15]:
def f_plot_metrics(df,col_list):
    
    plt.figure()
    for key in col_list:
        plt.plot(df_metrics[key],label=key,marker='*',linestyle='')
    plt.legend()
    
#     col_list=list(col_list)
#     df.plot(kind='line',x='step',y=col_list)
    
# f_plot_metrics(df_metrics,['spec_chi','hist_chi'])

interact_manual(f_plot_metrics,df=fixed(df_metrics), col_list=SelectMultiple(options=df_metrics.columns.values))

interactive(children=(SelectMultiple(description='col_list', options=('step', 'epoch', 'Dreal', 'Dfake', 'Dful…

<function __main__.f_plot_metrics(df, col_list)>

In [17]:

chi=df_metrics.quantile(q=0.2,axis=0)['hist_chi']
print(chi)
df_metrics[(df_metrics['hist_chi']<=chi)&(df_metrics.epoch>1)].sort_values(by=['hist_chi']).head(10)

-0.9067538738250733


,step,epoch,Dreal,Dfake,Dfull,G_adv,G_full,spec_loss,hist_loss,spec_chi,hist_chi,D(x),D_G_z1,D_G_z2,time
40558,40558.0,8.0,0.132478,0.187254,0.319731,6.746446,115.549171,108.802727,-0.723210,109.847122,-2.324598,2.724129,-6.359318,-6.744827,0.471375
40562,40562.0,8.0,0.175614,0.147250,0.322863,4.069386,114.584091,110.514702,-1.451975,109.452179,-2.305127,3.575622,-3.998980,-4.049746,0.473290
33553,33553.0,7.0,0.168974,0.200229,0.369203,6.509551,116.814972,110.305420,-0.911502,109.785095,-2.272750,2.696006,-6.338140,-6.507777,0.476615
34330,34330.0,7.0,0.149098,0.125781,0.274880,3.619483,115.330429,111.710945,-1.237876,109.753532,-2.270606,3.406135,-3.168069,-3.586084,0.460814
31229,31229.0,6.0,0.172381,0.151311,0.323692,4.210439,112.901123,108.690681,-0.906113,109.604248,-2.259225,3.695278,-3.130236,-4.192879,0.447018
40524,40524.0,8.0,0.167132,0.154666,0.321797,4.752157,113.093857,108.341698,-1.086549,108.918808,-2.251692,3.409426,-2.943850,-4.742093,0.469023
36728,36728.0,7.0,0.185567,0.146407,0.331975,3.861784,111.937027,108.075241,-0.439015,109.643478,-2.244315,4.146925,-3.530289,-3.835701,0.461702
34319,34319.0,7.0,0.199543,0.163012,0.362555,5.033240,116.070259,111.037018,-0.524088,107.853500,-2.241580,5.981313,-4.209482,-5.025881,0.551857
38900,38900.0,8.0,0.195646,0.146350,0.341996,3.539005,113.092621,109.553619,-0.580611,108.901672,-2.228121,3.604151,-3.708694,-3.507765,0.502798
35483,35483.0,7.0,0.137304,0.138348,0.275652,5.395416,114.045074,108.649658,-0.128622,109.848801,-2.225577,3.102691,-2.806516,-5.389395,0.528692


In [18]:
# display(df_metrics.sort_values(by=['hist_chi']).head(8))
# display(df_metrics.sort_values(by=['spec_chi']).head(8))

<!-- ### Read validation data -->

## Read stored chi-squares for images

In [19]:
## Get sigma list from saved files
flist=glob.glob(result_dir+'/df_processed*')
sigma_lst=[i.split('/')[-1].split('df_processed_')[-1].split('.pkle')[0] for i in flist]
sigma_lst.sort() ### Sorting is important for labels to match !!

labels_lst=np.arange(len(sigma_lst))

In [20]:
sigma_lst,labels_lst

(['0.5', '0.8', '1.1'], array([0, 1, 2]))

In [21]:
### Create a merged dataframe

df_list=[]
for label in labels_lst:
    df=pd.read_pickle(result_dir+'/df_processed_{0}.pkle'.format(str(sigma_lst[label])))
    df[['epoch','step']]=df[['epoch','step']].astype(int)
    df['label']=df.epoch.astype(str)+'-'+df.step.astype(str) # Add label column for plotting
    df_list.append(df)

for i,df in enumerate(df_list):
    df1=df.add_suffix('_'+str(i))
    # renaming the columns to be joined on
    keys=['epoch','step','img_type','label']
    rename_cols_dict={key+'_'+str(i):key for key in keys}
#     print(rename_cols_dict)
    df1.rename(columns=rename_cols_dict,inplace=True) 
    df_list[i]=df1
    
df_merged=reduce(lambda x, y : pd.merge(x, y, on = ['step','epoch','img_type','label']), df_list)

### Get sum of all 4 classes for 3 types of chi-squares
for chi_type in ['chi_1','chi_spec1','chi_1c']:
    keys=[chi_type+'_'+str(label) for label in labels_lst]
#     display(df_merged[keys].sum(axis=1))
    df_merged['sum_'+chi_type]=df_merged[keys].sum(axis=1)
del df_list



In [22]:

def f_plot_hist_spec(df,param_labels,sigma_lst,steps_list,bkg_dict,plot_type):

    img_size=64
    assert plot_type in ['hist','spec','grid','spec_relative'],"Invalid mode %s"%(plot_type)

    if plot_type in ['hist','spec','spec_relative']:     fig=plt.figure(figsize=(6,6))
    for par_label in param_labels:
        df=df[df.step.isin(steps_list)]
#         print(df.shape)
        idx=sigma_lst.index(par_label)
        suffix='_%s'%(idx)
        dict_bkg=bkg_dict[str(par_label)]
        
        for (i,row),marker in zip(df.iterrows(),itertools.cycle('>^*sDHPdpx_')):
            label=row.label+'_'+str(par_label)
            if plot_type=='hist':
                x1=row['hist_bin_centers'+suffix]
                y1=row['hist_val'+suffix]
                yerr1=row['hist_err'+suffix]
                x1=f_invtransform(x1)
                
                plt.errorbar(x1,y1,yerr1,marker=marker,markersize=5,linestyle='',label=label)
            if plot_type=='spec':
                y2=row['spec_val'+suffix]
                yerr2=row['spec_sdev'+suffix]/np.sqrt(row['num_imgs'+suffix])
                x2=np.arange(len(y2))

                plt.fill_between(x2, y2 - yerr2, y2 + yerr2, alpha=0.4)
                plt.plot(x2, y2, marker=marker, linestyle=':',label=label)

            if plot_type=='spec_relative':

                y2=row['spec_val'+suffix]
                yerr2=row['spec_sdev'+suffix]
                x2=np.arange(len(y2))

                ### Reference spectrum
                y1,yerr1=dict_bkg['spec_val'],dict_bkg['spec_sdev']
                y=y2/y1
                ## Variance is sum of variance of both variables, since they are uncorrelated

                # delta_r= |r| * sqrt(delta_a/a)^2 +(\delta_b/b)^2) / \sqrt(N)
                yerr=(np.abs(y))*np.sqrt((yerr1/y1)**2+(yerr2/y2)**2)/np.sqrt(row['num_imgs'+suffix])
                
                plt.fill_between(x2, y - yerr, y + yerr, alpha=0.4)
                plt.plot(x2, y, marker=marker, linestyle=':',label=label)

            if plot_type=='grid':
                images=np.load(row.fname)[:,:,:,0]
                print(images.shape)
                f_plot_grid(images[:18],cols=6,fig_size=(10,5))
            
        ### Plot reference data
        if plot_type=='hist':
            x,y,yerr=dict_bkg['hist_bin_centers'],dict_bkg['hist_val'],dict_bkg['hist_err']
            x=f_invtransform(x)
            plt.errorbar(x, y,yerr,color='k',linestyle='-',label='bkgnd')   
            plt.title('Pixel Intensity Histogram')
            plt.xscale('symlog',linthreshx=50)
        
        if plot_type=='spec':
            y,yerr=dict_bkg['spec_val'],dict_bkg['spec_sdev']/np.sqrt(num_bkgnd)
            x=np.arange(len(y))
            plt.fill_between(x, y - yerr, y + yerr, color='k',alpha=0.8)
            plt.title('Spectrum')
            plt.xlim(0,img_size/2)

        if plot_type=='spec_relative':
            plt.axhline(y=1.0,color='k',linestyle='-.')
            plt.title("Relative spectrum")
            plt.xlim(0,img_size/2)
            plt.ylim(0.5,2)

    if plot_type in ['hist','spec']:     
        plt.yscale('log')
    plt.legend(bbox_to_anchor=(0.3, 0.75),ncol=2, fancybox=True, shadow=True,prop={'size':6})


# f_plot_hist_spec(df_merged,[sigma_lst[-1]],sigma_lst,[best_step[0]],bkgnd_dict,'hist')

### Slice best steps

In [23]:
def f_slice_merged_df(df,cutoff=0.2,sort_col='chi_1',col_mode='all',label='all',params_lst=[0,1,2],head=10,epoch_range=[0,None],use_sum=True,display_flag=False):
    ''' View dataframe after slicing
    '''

    if epoch_range[1]==None: epoch_range[1]=df.max()['epoch']
    df=df[(df.epoch<=epoch_range[1])&(df.epoch>=epoch_range[0])]

    ######### Apply cutoff to keep reasonable chi1 and chispec1
    #### Add chi-square columns to use
    chi_cols=[]
    if use_sum: ## Add sum chi-square columns
        for j in ['chi_1','chi_spec1','chi_1c']: chi_cols.append('sum_'+j)
        
    if label=='all': ### Add chi-squares for all labels
        for j in ['chi_1','chi_spec1','chi_1c']:
            for idx,i in enumerate(params_lst): chi_cols.append(j+'_'+str(idx))
    else: ## Add chi-square for specific label
        assert label in params_lst, "label %s is not in %s"%(label,params_lst)
        label_idx=params_lst.index(label)
        print(label_idx)
        for j in ['chi_1','chi_spec1','chi_1c']: chi_cols.append(j+'_'+str(label_idx))
#     print(chi_cols)
    q_dict=dict(df_merged.quantile(q=cutoff,axis=0)[chi_cols])
    # print(q_dict)
    strg=['%s < %s'%(key,q_dict[key]) for key in chi_cols ]
    query=" & ".join(strg)
    # print(query)
    df=df.query(query)
    
    # Sort dataframe
    df1=df[df.epoch>0].sort_values(by=sort_col)
    chis=[i for i in df_merged.columns if 'chi' in i]
    col_list=['label']+chis+['epoch','step']
    if (col_mode=='short'): 
        col_list=['label']+[i for i in df_merged.columns if i.startswith('sum')]
        col_list=['label']+chi_cols
    df2=df1.head(head)[col_list]
    
    if display_flag: display(df2) # Display df
    
    return df2

# f_slice_merged_df(df_merged,cutoff=0.3,sort_col='sum_chi_1',label=0.65,params_lst=[0.5,0.65,0.8,1.1],use_sum=True,head=2000,display_flag=False,epoch_range=[7,None])

In [24]:
cols_to_sort=np.unique([i for i in df_merged.columns for j in ['chi_1_','chi_spec1_'] if ((i.startswith(j)) or (i.startswith('sum')))])

w=interactive(f_slice_merged_df,df=fixed(df_merged),
cutoff=widgets.FloatSlider(value=0.3, min=0, max=1.0, step=0.01), 
col_mode=['all','short'], display_flag=widgets.Checkbox(value=False),
use_sum=widgets.Checkbox(value=True),
label=ToggleButtons(options=['all']+sigma_lst), params_lst=fixed(sigma_lst),
head=widgets.IntSlider(value=10,min=1,max=20,step=1),
epoch_range=widgets.IntRangeSlider(value=[0,np.max(df.epoch.values)],min=0,max=np.max(df.epoch.values),step=1),
sort_col=cols_to_sort
)
display(w)

interactive(children=(FloatSlider(value=0.3, description='cutoff', max=1.0, step=0.01), Dropdown(description='…

In [25]:
df_sliced=w.result
# df_sliced

In [26]:
best_step=[]

best_step.append(f_slice_merged_df(df_merged,cutoff=0.9,sort_col='sum_chi_1',label='all',use_sum=True,head=4,display_flag=False,epoch_range=[7,None],params_lst=sigma_lst).step.values)
best_step.append(f_slice_merged_df(df_merged,cutoff=0.9,sort_col='sum_chi_spec1',label='all',use_sum=True,head=4,display_flag=False,epoch_range=[7,None],params_lst=sigma_lst).step.values)
best_step.append(f_slice_merged_df(df_merged,cutoff=0.9,sort_col='sum_chi_1c',label='all',use_sum=True,head=2,display_flag=False,epoch_range=[7,None],params_lst=sigma_lst).step.values)

# best_step.append([46669,34281])
best_step=np.unique([i for j in best_step for i in j])
print(best_step)
best_step

[36130 37350 37500 37820 40790 41370 41770 42870 44300 45790]


array([36130, 37350, 37500, 37820, 40790, 41370, 41770, 42870, 44300,
       45790])

In [27]:
# best_step=[6176]
# best_step= [32300, 35810, 36020, 37030, 38640, 42480, 43850]

# best_step=np.arange(40130,40135).astype(int)

In [28]:
df_best=df_merged[df_merged.step.isin(best_step)]
print(df_best.shape)
print([(df_best[df_best.step==step].epoch.values[0],df_best[df_best.step==step].step.values[0]) for step in best_step])
# print([(df_best.loc[idx].epoch,df_best.loc[idx].step) for idx in best_idx])

(10, 61)
[(7, 36130), (7, 37350), (8, 37500), (8, 37820), (8, 40790), (8, 41370), (8, 41770), (9, 42870), (9, 44300), (9, 45790)]


In [29]:
col_list=['label']+[i for i in df_merged.columns if i.startswith('sum')]

df_best[col_list]

,label,sum_chi_1,sum_chi_spec1,sum_chi_1c
3612,7-36130,0.283439,29.608761,0.003751
3734,7-37350,0.112545,21.431770,0.009122
3749,8-37500,0.252598,21.074777,0.003458
3781,8-37820,0.114074,28.493671,0.013794
4078,8-40790,0.082683,25.700527,0.006913
4136,8-41370,0.671015,16.361825,0.053157
4176,8-41770,0.448458,16.070034,0.008436
4286,9-42870,0.339939,15.848809,0.010587
4429,9-44300,0.089749,22.711669,0.007462
4578,9-45790,0.562333,16.485714,0.018261


### Interactive plot

In [30]:
interact_manual(f_plot_hist_spec,df=fixed(df_merged),
                param_labels=SelectMultiple(options=sigma_lst),sigma_lst=fixed(sigma_lst),
                steps_list=SelectMultiple(options=best_step),
                bkg_dict=fixed(bkgnd_dict),plot_type=ToggleButtons(options=['hist','spec','grid','spec_relative']))

interactive(children=(SelectMultiple(description='param_labels', options=('0.5', '0.8', '1.1'), value=()), Sel…

<function __main__.f_plot_hist_spec(df, param_labels, sigma_lst, steps_list, bkg_dict, plot_type)>

In [ ]:
# ### Check deterministic
# main_dir='/global/cfs/cdirs/m3363/vayyar/cosmogan_data/results_from_other_code/pytorch/results/128sq/'
# epoch,step=0,230
# f1='20210113_185116_cgan_predict_0.65_m2/images/gen_img_label-0.5_epoch-{0}_step-{1}.npy'.format(epoch,step)
# f2='20210113_092234_cgan_predict_0.65_m2/images/gen_img_label-0.5_epoch-{0}_step-{1}.npy'.format(epoch,step)

# epoch,step=3,20
# f1='20210114_191648_nb_test/images/gen_img_label-0.5_epoch-{0}_step-{1}.npy'.format(epoch,step)
# f2='20210114_193009_nb_test/images/gen_img_label-0.5_epoch-{0}_step-{1}.npy'.format(epoch,step)

# a1=np.load(main_dir+f1)
# a2=np.load(main_dir+f2)
# # print(a1.shape,a2.shape)

# print(np.mean(a1),np.mean(a2))
# print(np.max(a1),np.max(a2))

### Delete unwanted stored models
(Since deterministic runs aren't working well )

In [113]:
# fldr='/global/cfs/cdirs/m3363/vayyar/cosmogan_data/results_from_other_code/pytorch/results/128sq/20210119_134802_cgan_predict_0.65_m2/models'
fldr=result_dir
print(fldr)
flist=glob.glob(fldr+'/models/checkpoint_*.tar')
len(flist)

/global/cfs/cdirs/m3363/vayyar/cosmogan_data/results_from_other_code/pytorch/results/3d_cGAN/20210227_050213_3dcgan_predict_0.65_m2


3742

In [31]:
# # Delete unwanted stored images
# for i in flist:
#     try:
#         step=int(i.split('/')[-1].split('_')[-1].split('.')[0])
#         if step not in best_step:
# #             print("Found")
# #             print(step)
#             os.remove(i)
#             pass
#         else: 
#             print(step)
# #             print(i)
#     except Exception as e:
# #         print(e)
# #         print(i)
#         pass

In [118]:
best_step

array([27600, 30950, 42620, 43830, 50440, 58320, 58330, 59500, 64670,
       65600])

In [119]:
np.load('/global/cfs/cdirs/m3363/vayyar/cosmogan_data/raw_data/3d_data/dataset2a_3dcgan_4univs_64cube_simple_splicing/norm_1_sig_0.5_train_val.npy',mmap_mode='r').shape

(14376, 1, 64, 64, 64)